<a href="https://colab.research.google.com/github/luislauriano/Data_Science/blob/master/Projeto_construcao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Explicar toda etapa da API do Spotify**

In [12]:
!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv
 
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id="dcffc14978ed497aa58834a7864a99b7", client_secret="e8c5cc78c8e74ccd92323d146c635b47"))

In [0]:
def get_artist(name):
    results = sp.search(name)
    items = results['tracks']['items']
    if len(items) > 0:
        return items[0]['artists'][0]
    else:
        return None

In [0]:
def get_artist_id(artist):
    return artist['id']

In [0]:
def get_artist_albums_id_names(id):
  albums = sp.artist_albums(id, country = 'BR', limit=3)
  albums_id_name = {}
  for i in range(len(albums['items'])):
    id = albums['items'][i]['id']
    name = albums['items'][i]['name']
    albums_id_name[id] = name
 
  return albums_id_name

In [0]:
def get_album_songs(album_id, album_name):
  spotify_album = {}
 
  tracks = sp.album_tracks(album_id)
  
  for n in range(len(tracks['items'])):
    id_track = tracks['items'][n]['id']
    track = sp.track(id_track)
    spotify_album[id_track] = {}
    
    spotify_album[id_track]['album'] = album_name
    spotify_album[id_track]['album_type'] = track['album']['album_type']
    spotify_album[id_track]['track_number'] = track['track_number']
    spotify_album[id_track]['id_track'] = track['id']
    spotify_album[id_track]['name'] = track['name']
    spotify_album[id_track]['popularity'] = track['popularity']
    spotify_album[id_track]['explicit'] = track['explicit']
    spotify_album[id_track]['duration_ms'] = track['duration_ms']
    spotify_album[id_track]['release_date'] = track['album']['release_date']
 
    artists_track = track['artists']
    spotify_album[id_track]['artists'] = []
    for artist in artists_track:
      spotify_album[id_track]['artists'].append(artist['name'])
  return spotify_album


In [0]:
def get_all_albums_songs(albums_ids_names):
  spotify_albums = []
  albums_names = []
  for id, name in albums_ids_names.items():
    if name not in albums_names:
      albums_names.append(name)
      album_songs = get_album_songs(id,name) 
    for item in album_songs.items():
      spotify_albums.append(item[1]) 
  return spotify_albums

In [0]:
def convert_to_csv(filepath, name):
  keys = filepath[0].keys()
  print(keys)
  csv_name = ''+ name + '.csv'
  with open(csv_name, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(filepath)
  return

In [19]:
name = "NexoAnexo"
artist = get_artist(name)    
if artist:
  artist_id = get_artist_id(artist)
  albums_id_names = get_artist_albums_id_names(artist_id)
  all_albums = get_all_albums_songs(albums_id_names)
  convert_to_csv(all_albums, 'NexoAnexo')   
else:
  logger.error("Can't find artist: %s", artist)

dict_keys(['album', 'album_type', 'track_number', 'id_track', 'name', 'popularity', 'explicit', 'duration_ms', 'release_date', 'artists'])


## **Agora vamos visualizar os dados do meu mano NexoAnexo**

Colocar todas as etapas do projeto de lol 

Depois de organizar o que gostaríamos de fazer e mexer na API do Spotify para ter um .csv bonito, nos falta apenas as visualizações dos dados. Porém, entretanto, todavia, ainda não disse quais perguntas vamos responder. Então vamos lá:

Qual é a música mais popular de Nexo?
Qual é o álbum mais popular?
E o single?

Apostas: 

No album Real Plug, 
No album Trap de Cria, acredito que seja OI Fake 
No album TFF, acredito que seja Match, por mais que a minha preferida seja ...

Musica mais pouplar, Ratatah 
Album mais popular, TFF


Ver se a popularidade está funcionando da maneira correta

In [0]:
import altair as alt
import pandas as pd
 
df = pd.read_csv('NexoAnexo.csv')

In [30]:
df.head()

,album,album_type,track_number,id_track,name,popularity,explicit,duration_ms,release_date,artists
0,Trap from Future,album,1,6Mx214YlNnbj7un9PrvmGi,Vem No Tum Tum,18,True,216917,2020-03-13,['NexoAnexo']
1,Trap from Future,album,2,1xOs8YJkkKm5HhRiP1ha4g,Drip Know Me,3,True,154646,2020-03-13,"['NexoAnexo', 'Drippy Kid Jay']"
2,Trap from Future,album,3,4s4G0Z6mSkrauIvBXRlVHk,Match,5,True,180716,2020-03-13,['NexoAnexo']
3,Trap from Future,album,4,5C1iQCV3xuc0oHQpe7fhhQ,Trap & Brega,7,True,164468,2020-03-13,['NexoAnexo']
4,Trap from Future,album,5,3H2TfCpxaSfEh5nVbcDzrp,Marca Sem Roupa,6,True,148364,2020-03-13,"['NexoAnexo', 'Victor KR']"


In [51]:
df[['name', 'popularity']].sort_values(ascending=False, by ='popularity')

,name,popularity
0,Vem No Tum Tum,18
15,Trap de Cria,9
3,Trap & Brega,7
4,Marca Sem Roupa,6
2,Match,5
12,Fogo No Incenso,5
10,Como Tem Que Ser,4
23,Isso Que É Foda,4
32,2020,4
6,Não Posso Morrer Novo,4


In [50]:
data_top20 = df.sort_values('popularity', ascending = False).head(20)

alt.Chart(data_top20).mark_circle(color = '#222437', opacity = 1).encode(
    y = alt.Y('name', axis = alt.Axis(title='Nome'), sort = None),
    x = alt.X('popularity', axis = alt.Axis(title = "Popularidade")),
    tooltip = ['name', 'popularity', 'album', 'album_type']
)

alt.Chart(...)

Lembrem da definição de popularidade que foi dada lá no volume 01. Uma música nova tem muito mais chance de ser mais popular. 

O sucesso Trap de cria